## Cálculo del Estimated RTT utilizando Exponential Weighted Moving Averarge (EWMA)
### Juan Luis Ávila, Guillermo Ribero, Juan José Caballero
Este "notebook" está diseñado para que se grafiquen los valores de 'Sample RTT' y 'Estimated RTT' de un determinado "host". Los datos de 'Sample RTT' pueden ser calculados de manera aleatoria o utilizando un archivo csv. Para utilizar la herramienta que hemos creado es necesario que modifique el código en ningún momento, sino que por el contrario, escoja la opción de cómo quiere escoger los valores de 'Sample RTT' con los botones que están al final y con la barra deslizante pueda elegir el valor de la constante $\alpha$. Para que estos widgets aparezcan simplemente ejecute la celda de abajo.

In [92]:
# Importación de paquetes
import random as rd
import matplotlib.pyplot as plt
import pandas as pd
from tkinter import Tk     
from tkinter.filedialog import askopenfilename
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

#Función que calcula el 'Estimated RTT'
def ewma_calculation(csv=False,alpha=0.125):
    if not csv:
        SampleRTT = []
        time = []
        for i in range(200):
            SampleRTT.append(rd.randrange(150,351))
            time.append(i)
    else:
        my_csv = pd.read_csv(csv)
        SampleRTT = my_csv['RTT'].tolist()
        time = my_csv['time'].tolist()

    ewma = [SampleRTT[0]]
    for i in range(1,len(SampleRTT)):
        ewma.append((1-alpha)*ewma[i-1]+alpha*SampleRTT[i])
    plt.plot(time,SampleRTT, label = 'SampleRTT')
    plt.plot(time,ewma, label = 'EstimatedRTT')
    plt.xlabel('Time(s)')
    plt.ylabel('RTT (ms)')
    plt.legend()
    plt.show()
    return ewma

# Interfaz gráfica - Creación de Widgets amigables
boton = widgets.RadioButtons(
        options=['Importar datos desde un archivo csv.', 'Datos aleatorios.'],
        #layout={'width': 'max-content'}, # If the items' names are long
        disabled=False,
        value = None
)

option = widgets.Box([
    widgets.Label(value='¿Qué datos quiere utilizar para hacer el cálculo del EWMA?'),
    boton
])
output = widgets.Output()
alpha_widget = widgets.FloatSlider(
    name = 'alpha',
    value=0.125,
    min=0,
    max=1,
    step=0.005,
    description='Alpha:',
)
csv =''
def on_value_change(change):
    with output:
        output.clear_output()
        if boton.value == 'Importar datos desde un archivo csv.':
            global csv
            Tk().withdraw()
            csv = askopenfilename(filetypes = (('Archivos CSV', '*.csv'),))
            ewma_calculation(csv)
        elif boton.value == 'Datos aleatorios.':
            ewma_calculation()
def change_alpha(alpha):
    with output:
        output.clear_output()
        if boton.value == 'Importar datos desde un archivo csv.':
            ewma_calculation(csv,alpha['new'])
        elif boton.value == 'Datos aleatorios.':
            ewma_calculation(alpha=alpha['new'])
    
display(option)
display(alpha_widget,output)
boton.observe(on_value_change,names='value')
alpha_widget.observe(change_alpha,names='value')

Box(children=(Label(value='¿Qué datos quiere utilizar para hacer el cálculo del EWMA?'), RadioButtons(options=…

FloatSlider(value=0.125, description='Alpha:', max=1.0, step=0.005)

Output()